# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'BGM)/NNKCAQEDOSALS S(TATEEAGLR  L', 'calculationPrice': 'close', 'open': None, 'openTime': None, 'openSource': 'ifcfaiol', 'close': None, 'closeTime': None, 'closeSource': 'lioifafc', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 134.14, 'latestSource': 'Close', 'latestTime': 'April 8, 2021', 'latestUpdate': 1659076670862, 'latestVolume': None, 'iexRealtimePrice': 136.39, 'iexRealtimeSize': 101, 'iexLastUpdated': 1676538788891, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 133.6, 'previousVolume': 85797718, 'change': 2.56, 'changePercent': 0.02012, 'volume': None, 'iexMarketPercent': 27.368231240342794, 'iexVolume': 1604224, 'avgTotalVolume': 105264383, 'iexBidPrice': 0,

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

35.46

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index=my_columns
            ),
            ignore_index=True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,133.32,52.26,N/A
1,AAL,24.60,-1.33,N/A
2,AAP,183.10,25.82,N/A
3,AAPL,136.29,35.83,N/A
4,ABBV,108.80,40.9,N/A
...,...,...,...,...
500,YUM,116.73,40,N/A
501,ZBH,168.12,-251.6,N/A
502,ZBRA,525.00,55.95,N/A
503,ZION,56.00,8.69,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe50 = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe50 = final_dataframe50[:50]
final_dataframe50.reset_index(inplace=True)
final_dataframe50.drop('index', axis=1, inplace=True)
final_dataframe50

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,620.09,4.9,N/A
1,ALL,121.93,6.98,N/A
2,UNM,27.63,7,N/A
3,AFL,52.85,7.76,N/A
4,EBAY,62.49,8.03,N/A
5,COO,394.09,8.6,N/A
6,ZION,56.00,8.69,N/A
7,KIM,19.94,8.77,N/A
8,CE,150.10,9.03,N/A
9,PGR,95.81,10.01,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe50.index)
for row in final_dataframe50.index:
    final_dataframe50.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe50.loc[row, 'Price'])

final_dataframe50

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,620.09,4.9,322
1,ALL,121.93,6.98,1640
2,UNM,27.63,7,7238
3,AFL,52.85,7.76,3784
4,EBAY,62.49,8.03,3200
5,COO,394.09,8.6,507
6,ZION,56.00,8.69,3571
7,KIM,19.94,8.77,10030
8,CE,150.10,9.03,1332
9,PGR,95.81,10.01,2087


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
print(data['AAPL']['advanced-stats'])
print(data['AAPL']['advanced-stats']['enterpriseValue'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda =  enterprise_value / ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value / gross_profit

{'beta': 1.2816466786507301, 'totalCash': 77992140274, 'currentDebt': 132571326938, 'revenue': 304637865082, 'grossProfit': 114693950895, 'totalRevenue': 300800801644, 'EBITDA': 88500574224, 'revenuePerShare': 17.75, 'revenuePerEmployee': 2087052.98, 'debtToEquity': 5.601432482505921, 'profitMargin': 0.22436970997677988, 'enterpriseValue': 2315350318484, 'enterpriseValueToRevenue': 7.73, 'priceToSales': 7.62, 'priceToBook': 33.62, 'forwardPERatio': 30.6020840021368, 'pegRatio': 2.0467885467194655, 'peHigh': 40.90714164429352, 'peLow': 17.030251616979676, 'week52highDate': '2021-01-18', 'week52lowDate': '2020-04-06', 'putCallRatio': 0.3771286730275761, 'companyName': 'Apple Inc', 'marketcap': 2254829105107, 'week52high': 149.13, 'week52low': 65.93, 'week52highSplitAdjustOnly': 147.07, 'week52highDateSplitAdjustOnly': '2021-01-24', 'week52lowSplitAdjustOnly': 68.62, 'week52lowDateSplitAdjustOnly': '2020-04-21', 'week52change': 0.9939912033534105, 'sharesOutstanding': 17472280644, 'float'

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earning Ratio',
    'PE Percentile',
    'Price-To-Book Ratio',
    'PB Percentile',
    'Price-To-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = enterprise_value / gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN

        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
            ],
            index=rv_columns),
            ignore_index=True
        )

In [13]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,135.83,N/A,51.54,N/A,8.66,N/A,7.5,N/A,33.008589,N/A,13.754825,N/A,N/A
1,AAL,23.99,N/A,-1.29,N/A,-2.28,N/A,0.9065,N/A,-4.393689,N/A,2.288821,N/A,N/A
2,AAP,186.40,N/A,26.08,N/A,3.4,N/A,1.21,N/A,12.333715,N/A,2.740697,N/A,N/A
3,AAPL,133.47,N/A,37.16,N/A,33.48,N/A,7.53,N/A,25.985781,N/A,19.326824,N/A,N/A
4,ABBV,109.50,N/A,39.75,N/A,14.8,N/A,4.28,N/A,13.587634,N/A,8.500582,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.75,N/A,39,N/A,-4.51,N/A,6.34,N/A,25.309100,N/A,10.924302,N/A,N/A
501,ZBH,168.29,N/A,-253.57,N/A,2.87,N/A,4.92,N/A,31.336280,N/A,8.233703,N/A,N/A
502,ZBRA,509.30,N/A,54.21,N/A,13.15,N/A,6.34,N/A,37.025007,N/A,13.937949,N/A,N/A
503,ZION,57.00,N/A,8.86,N/A,1.24,N/A,2.69,N/A,6.744523,N/A,2.931931,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,244.00,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
42,APA,17.67,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,270.32,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12.00,N/A,9.91,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,36.32,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.33,N/A,14.61,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,61.00,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,35.67,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,177.01,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2360.06,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for column in ['Price-to-Earning Ratio', 'Price-To-Book Ratio', 'Price-To-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,135.83,N/A,51.54,N/A,8.66,N/A,7.5000,N/A,33.008589,N/A,13.754825,N/A,N/A
1,AAL,23.99,N/A,-1.29,N/A,-2.28,N/A,0.9065,N/A,-4.393689,N/A,2.288821,N/A,N/A
2,AAP,186.40,N/A,26.08,N/A,3.40,N/A,1.2100,N/A,12.333715,N/A,2.740697,N/A,N/A
3,AAPL,133.47,N/A,37.16,N/A,33.48,N/A,7.5300,N/A,25.985781,N/A,19.326824,N/A,N/A
4,ABBV,109.50,N/A,39.75,N/A,14.80,N/A,4.2800,N/A,13.587634,N/A,8.500582,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.75,N/A,39.00,N/A,-4.51,N/A,6.3400,N/A,25.309100,N/A,10.924302,N/A,N/A
501,ZBH,168.29,N/A,-253.57,N/A,2.87,N/A,4.9200,N/A,31.336280,N/A,8.233703,N/A,N/A
502,ZBRA,509.30,N/A,54.21,N/A,13.15,N/A,6.3400,N/A,37.025007,N/A,13.937949,N/A,N/A
503,ZION,57.00,N/A,8.86,N/A,1.24,N/A,2.6900,N/A,6.744523,N/A,2.931931,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [25]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earning Ratio': 'PE Percentile',
    'Price-To-Book Ratio': 'PB Percentile',
    'Price-To-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric]) / 100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
459,UNM,28.47,7024,7.00,0.164356,0.5227,0.047525,0.4357,0.019802,3.605898,0.047525,0.420734,0.00396,5.663366
300,MCK,197.48,1012,-7.72,0.108911,-65.7300,0.00396,0.1304,0.005941,7.106681,0.065347,2.687213,0.106931,5.821782
194,FTI,7.78,25706,-1.09,0.156436,0.8689,0.067327,0.2727,0.009901,2.235173,0.043564,1.416426,0.045545,6.455446
18,AIG,47.03,4252,-6.90,0.118812,0.6122,0.053465,0.9256,0.079208,7.273819,0.069307,0.919304,0.019802,6.811881
304,MET,64.09,3120,10.91,0.192079,0.7289,0.055446,0.8102,0.061386,5.416333,0.051485,0.793101,0.013861,7.485149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,IDXX,510.68,N/A,77.93,0.918812,70.9600,0.974257,16.3800,0.964356,56.182417,0.946535,27.922386,0.976238,95.60396
150,DXCM,396.46,N/A,75.03,0.910891,20.7300,0.922772,19.4000,0.982178,96.437635,0.990099,28.059048,0.978218,95.683168
345,NVDA,574.80,N/A,85.00,0.930693,21.0700,0.930693,22.3400,0.990099,62.830326,0.958416,33.571276,0.99604,96.118812
358,PAYC,381.17,N/A,161.18,0.970297,36.2600,0.958416,28.1300,0.99802,95.160392,0.986139,31.786579,0.992079,98.09901


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [26]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
459,UNM,28.47,7024,7.00,0.164356,0.5227,0.047525,0.4357,0.019802,3.605898,0.047525,0.420734,0.00396,0.056634
300,MCK,197.48,1012,-7.72,0.108911,-65.7300,0.00396,0.1304,0.005941,7.106681,0.065347,2.687213,0.106931,0.058218
194,FTI,7.78,25706,-1.09,0.156436,0.8689,0.067327,0.2727,0.009901,2.235173,0.043564,1.416426,0.045545,0.064554
18,AIG,47.03,4252,-6.90,0.118812,0.6122,0.053465,0.9256,0.079208,7.273819,0.069307,0.919304,0.019802,0.068119
304,MET,64.09,3120,10.91,0.192079,0.7289,0.055446,0.8102,0.061386,5.416333,0.051485,0.793101,0.013861,0.074851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,IDXX,510.68,N/A,77.93,0.918812,70.9600,0.974257,16.3800,0.964356,56.182417,0.946535,27.922386,0.976238,0.95604
150,DXCM,396.46,N/A,75.03,0.910891,20.7300,0.922772,19.4000,0.982178,96.437635,0.990099,28.059048,0.978218,0.956832
345,NVDA,574.80,N/A,85.00,0.930693,21.0700,0.930693,22.3400,0.990099,62.830326,0.958416,33.571276,0.99604,0.961188
358,PAYC,381.17,N/A,161.18,0.970297,36.2600,0.958416,28.1300,0.99802,95.160392,0.986139,31.786579,0.992079,0.98099


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [27]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe50 = rv_dataframe[:50]
rv_dataframe50.reset_index(drop=True, inplace=True)
rv_dataframe50

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.47,7024,7.00,0.164356,0.5227,0.047525,0.4357,0.019802,3.605898,0.047525,0.420734,0.00396,0.056634
1,MCK,197.48,1012,-7.72,0.108911,-65.7300,0.00396,0.1304,0.005941,7.106681,0.065347,2.687213,0.106931,0.058218
2,FTI,7.78,25706,-1.09,0.156436,0.8689,0.067327,0.2727,0.009901,2.235173,0.043564,1.416426,0.045545,0.064554
3,AIG,47.03,4252,-6.90,0.118812,0.6122,0.053465,0.9256,0.079208,7.273819,0.069307,0.919304,0.019802,0.068119
4,MET,64.09,3120,10.91,0.192079,0.7289,0.055446,0.8102,0.061386,5.416333,0.051485,0.793101,0.013861,0.074851
5,AAL,23.99,8336,-1.29,0.153465,-2.2800,0.045545,0.9065,0.075248,-4.393689,0.039604,2.288821,0.081188,0.07901
6,ALL,121.23,1649,6.89,0.162376,1.3200,0.120792,0.8252,0.065347,2.561415,0.045545,0.804784,0.015842,0.08198
7,F,12.89,15515,-40.16,0.051485,1.6000,0.174257,0.3975,0.017822,6.283563,0.057426,2.998227,0.122772,0.084752
8,ABC,117.05,1708,-7.56,0.110891,-35.1000,0.011881,0.1217,0.00396,8.372408,0.10297,4.190444,0.207921,0.087525
9,DXC,31.77,6295,-2.90,0.136634,1.4200,0.146535,0.4448,0.021782,6.017609,0.055446,2.605784,0.10297,0.092673


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [28]:
portfolio_input()

In [29]:
position_size = float(portfolio_size) / len(rv_dataframe50.index)

for row in rv_dataframe50.index:
    rv_dataframe50.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe50.loc[row, 'Price'])

rv_dataframe50

d:\tutorial\algorithmic-trading-python\.virtualenv\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.47,7024,7.00,0.164356,0.5227,0.047525,0.4357,0.019802,3.605898,0.047525,0.420734,0.00396,0.056634
1,MCK,197.48,1012,-7.72,0.108911,-65.7300,0.00396,0.1304,0.005941,7.106681,0.065347,2.687213,0.106931,0.058218
2,FTI,7.78,25706,-1.09,0.156436,0.8689,0.067327,0.2727,0.009901,2.235173,0.043564,1.416426,0.045545,0.064554
3,AIG,47.03,4252,-6.90,0.118812,0.6122,0.053465,0.9256,0.079208,7.273819,0.069307,0.919304,0.019802,0.068119
4,MET,64.09,3120,10.91,0.192079,0.7289,0.055446,0.8102,0.061386,5.416333,0.051485,0.793101,0.013861,0.074851
5,AAL,23.99,8336,-1.29,0.153465,-2.2800,0.045545,0.9065,0.075248,-4.393689,0.039604,2.288821,0.081188,0.07901
6,ALL,121.23,1649,6.89,0.162376,1.3200,0.120792,0.8252,0.065347,2.561415,0.045545,0.804784,0.015842,0.08198
7,F,12.89,15515,-40.16,0.051485,1.6000,0.174257,0.3975,0.017822,6.283563,0.057426,2.998227,0.122772,0.084752
8,ABC,117.05,1708,-7.56,0.110891,-35.1000,0.011881,0.1217,0.00396,8.372408,0.10297,4.190444,0.207921,0.087525
9,DXC,31.77,6295,-2.90,0.136634,1.4200,0.146535,0.4448,0.021782,6.017609,0.055446,2.605784,0.10297,0.092673


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [30]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [31]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [32]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earning Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-To-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-To-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', float_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0],  column_formats[column][1])

d:\tutorial\algorithmic-trading-python\.virtualenv\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [ ]:
writer.save()